In [153]:
from pymongo import MongoClient
import pandas as pd
import json
import geojson
import os
import requests
from dotenv import load_dotenv
import time
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from folium import Icon
from folium import plugins

## VISUALIZING ##

In [74]:
client = MongoClient("localhost:27017")
db = client["cities_project"]

**IMPORTING JSON INTO DF**

In [75]:
def mongodb_to_df(name_collection):

    '''Function to get the collections from mongo and turn them to DF
    It takes 1 arg:
    - collection name.
    Also checks for duplicate columns and drops them, only based on lat and long. 
    '''

    df = pd.DataFrame(list(db[name_collection].find()))
    df_drop_duplicates = df.drop_duplicates(subset=df.columns.difference(["name"]))

    return df_drop_duplicates

    SAN FRANCISCO

In [76]:
SFstarbucks = mongodb_to_df("SFstarbucks")
SFbaskStadium = mongodb_to_df("SFbaskStadium")
SFschools = mongodb_to_df("SFschools")
SFairport = mongodb_to_df("SFairport")
SFtrainStations = mongodb_to_df("SFtrainStations")
SFpet_grooming = mongodb_to_df("SFpet_grooming")
SFdog_parks = mongodb_to_df("SFdog_parks")
SFbars_clubs = mongodb_to_df("SFbars_clubs")
SFvegan_vegetarian = mongodb_to_df("SFvegan_vegetarian")
SFvegan_vegetarian.head()

,_id,name,lat,lon
0,6547ac546d42dc580d4d6464,Ferry Building Marketplace,37.795536,-122.393425
1,6547ac546d42dc580d4d6465,Mission Dolores Park,37.759655,-122.426912
2,6547ac546d42dc580d4d6466,Berkeley Bowl,37.853335,-122.290239
3,6547ac546d42dc580d4d6467,Whole Foods,37.764381,-122.402680
4,6547ac546d42dc580d4d6468,Whole Foods,37.789980,-122.423402


In [77]:
'''def drop_rows_notStarb(dataframe):

    Function to make sure that all the caffes are starbucks. Because looking at the dataframe, that wan't happenning.
    Takes 1 arg:
    - Dataframe

    starbucks = dataframe['name'] == 'Starbucks'
    
    filtered_dataframe = dataframe[starbucks]
    
    return filtered_dataframe'''

"def drop_rows_notStarb(dataframe):\n\n    Function to make sure that all the caffes are starbucks. Because looking at the dataframe, that wan't happenning.\n    Takes 1 arg:\n    - Dataframe\n\n    starbucks = dataframe['name'] == 'Starbucks'\n    \n    filtered_dataframe = dataframe[starbucks]\n    \n    return filtered_dataframe"

In [78]:
'''SFstarbucks = drop_rows_notStarb(SFstarbucks)
SFstarbucks'''

'SFstarbucks = drop_rows_notStarb(SFstarbucks)\nSFstarbucks'

    NEW YORK

In [130]:
NYCstarbucks = mongodb_to_df("NYCstarbucks")
NYCbaskStadium = mongodb_to_df("NYCbaskStadium")
NYCschools = mongodb_to_df("NYCschools")
NYCairport = mongodb_to_df("NYCairport")
NYCtrainStations = mongodb_to_df("NYCtrain_stations")
NYCpet_grooming = mongodb_to_df("NYCpet_grooming")
NYCdog_parks = mongodb_to_df("NYCdog_parks")
NYCbars_clubs = mongodb_to_df("NYCbars_clubs")
NYCvegan_vegetarian = mongodb_to_df("NYCvegan_vegetarian")
NYCtrainStations.sample()

,_id,name,lat,lon
20,6547ad70a39b9f8989a37c62,Grand Central Terminal,40.752724,-73.97716


In [80]:
'''NYCstarbucks = drop_rows_notStarb(NYCstarbucks)
NYCstarbucks'''

'NYCstarbucks = drop_rows_notStarb(NYCstarbucks)\nNYCstarbucks'

    SEATTLE

In [136]:
STstarbucks = mongodb_to_df("STstarbucks")
STbaskStadium = mongodb_to_df("STbaskStadium")
STschools = mongodb_to_df("STschools")
STairport = mongodb_to_df("STairport")
STtrainStations = mongodb_to_df("STtrain_stations")
STpet_grooming = mongodb_to_df("STpet_grooming")
STdog_parks = mongodb_to_df("STdog_parks")
STbars_clubs = mongodb_to_df("STbars_clubs")
STvegan_vegetarian = mongodb_to_df("STvegan_vegetarian")


In [82]:
'''STstarbucks = drop_rows_notStarb(STstarbucks)
STstarbucks'''

'STstarbucks = drop_rows_notStarb(STstarbucks)\nSTstarbucks'

**START MAPPING**

First Import de collections from MongoDB

In [83]:
def mongodb_to_df_geo(name_collection):

    '''Function to get the collections from mongo and turn them to DF
    It takes 1 arg:
    - collection name. 
    '''
    json = db.get_collection(name_collection) 

    return json

    SAN FRANCISCO

In [107]:
SanFranciscoNeigh = mongodb_to_df_geo("SanFranciscoNeigh")

    NEW YORK

In [85]:
NewYorkNeigh = mongodb_to_df_geo("NewYorkNeigh")

    SEATTLE

In [86]:
SeattleNeigh = mongodb_to_df_geo("SeattleNeigh")

read geojson, because the collections from mongo don't work.

In [113]:
def read_geojson(relative_path):
    ''' Function to read geojson.
    It takes 1 arg:
    - Relative path of file.
    '''
    with open(relative_path) as f:
        return gpd.read_file(f)

In [116]:
SF_gj = read_geojson("../data/SanFrancisco.Neighborhoods.json")
NYC_gj = read_geojson("../data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
ST_gj = read_geojson("../data/Neighborhood_Map_Atlas_Neighborhoods.geojson")

Markers

LOCATIONS

In [97]:
san_francisco_loc = [37.75404,-122.44721]
new_york_loc = [40.79724,-73.94968]
seattle_loc = [47.60065,-122.31819]

Funcion para pins de cada tipo, despues llamar a estas funciones cuando se hace el mapa para tener todos los pins. 

## HEAT MAPS ##

In [154]:

def heat_map(name,loc,neigh_geojson, starbucks_df,stadium_df, schools_df, airport_df, trainstations_df, pet_grooming_df, dog_parks_df, bars_clubs_df,vegan_vegetarian_df):


    ''' Function to create heat maps with all the points of interest for each city.

    takes 12 arguments:
    - geojson of the city
    - the 9 dataframes of points of interests
    - name to save the heatmap
    - location.

    '''


    heatmap = folium.Map(location=loc, zoom_start=12)

    starbucks_coords = []
    stadium_coords = []
    school_coords = []
    trainstations_coords = []
    airport_coords = []
    pet_grooming_coords = []
    dog_parks_coords = []
    bars_clubs_coords = []
    vegan_vegetarian_coords = []


    geometry_starbucks = gpd.points_from_xy(starbucks_df['lon'], starbucks_df['lat'])
    starbucks_places = gpd.GeoDataFrame(starbucks_df, geometry=geometry_starbucks)

    for polygon in neigh_geojson.geometry:

        filtered_starbucks_places = starbucks_places[starbucks_places.geometry.within(polygon)]

        for _, row in filtered_starbucks_places.iterrows():
            starbucks_coords.append([row['lat'], row['lon']])

    geometry_stadium = gpd.points_from_xy(stadium_df['lon'], stadium_df['lat'])
    stadiums = gpd.GeoDataFrame(stadium_df, geometry=geometry_stadium)  

    for polygon in neigh_geojson.geometry:

        filtered_stadiums = stadiums[stadiums.geometry.within(polygon)]

        for _, row in filtered_stadiums.iterrows():
            stadium_coords.append([row['lat'], row['lon']])

    geometry_schools = gpd.points_from_xy(schools_df['lon'], schools_df['lat'])
    schools = gpd.GeoDataFrame(schools_df, geometry=geometry_schools)

    for polygon in neigh_geojson.geometry:

        filtered_schools = schools[schools.geometry.within(polygon)]

        for _, row in filtered_schools.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_airport = gpd.points_from_xy(airport_df['lon'], airport_df['lat'])
    airport = gpd.GeoDataFrame(airport_df, geometry=geometry_airport)

    for polygon in neigh_geojson.geometry:

        filtered_airport = airport[airport.geometry.within(polygon)]

        for _, row in filtered_airport.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_trainstations = gpd.points_from_xy(trainstations_df['lon'], trainstations_df['lat'])
    trainstations = gpd.GeoDataFrame(trainstations_df, geometry=geometry_trainstations)

    for polygon in neigh_geojson.geometry:

        filtered_trainstations = trainstations[trainstations.geometry.within(polygon)]

        for _, row in filtered_trainstations.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_pet_grooming = gpd.points_from_xy(pet_grooming_df['lon'], pet_grooming_df['lat'])
    pet_grooming = gpd.GeoDataFrame(pet_grooming_df, geometry=geometry_pet_grooming)

    for polygon in neigh_geojson.geometry:

        filtered_pet_grooming = pet_grooming[pet_grooming.geometry.within(polygon)]

        for _, row in filtered_pet_grooming.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_dog_parks = gpd.points_from_xy(dog_parks_df['lon'], dog_parks_df['lat'])
    dog_parks = gpd.GeoDataFrame(dog_parks_df, geometry=geometry_dog_parks)

    for polygon in neigh_geojson.geometry:

        filtered_dog_parks = dog_parks[dog_parks.geometry.within(polygon)]

        for _, row in filtered_dog_parks.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_bars_clubs = gpd.points_from_xy(bars_clubs_df['lon'], bars_clubs_df['lat'])
    bars_clubs = gpd.GeoDataFrame(bars_clubs_df, geometry=geometry_bars_clubs)

    for polygon in neigh_geojson.geometry:

        filtered_bars_clubs = bars_clubs[bars_clubs.geometry.within(polygon)]

        for _, row in filtered_bars_clubs.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_vegan_vegetarian = gpd.points_from_xy(vegan_vegetarian_df['lon'],vegan_vegetarian_df['lat'])
    vegan_vegetarian = gpd.GeoDataFrame(vegan_vegetarian_df, geometry=geometry_vegan_vegetarian)

    for polygon in neigh_geojson.geometry:

        filtered_vegan_vegetarian = vegan_vegetarian[vegan_vegetarian.geometry.within(polygon)]

        for _, row in filtered_vegan_vegetarian.iterrows():
            school_coords.append([row['lat'], row['lon']])

        HeatMap(starbucks_coords).add_to(heatmap)

        HeatMap(stadium_coords).add_to(heatmap)

        HeatMap(school_coords).add_to(heatmap)

        HeatMap(airport_coords).add_to(heatmap)

        HeatMap(trainstations_coords).add_to(heatmap)

        HeatMap(pet_grooming_coords).add_to(heatmap)

        HeatMap(dog_parks_coords).add_to(heatmap)

        HeatMap(bars_clubs_coords).add_to(heatmap)

        HeatMap(vegan_vegetarian_coords).add_to(heatmap)
    
        heatmap.save(f"figures/heatmap{name}.html")

        return heatmap

    SAN FRANCISCO

In [155]:
SFheatmap = heat_map("SFheatmap",san_francisco_loc, SF_gj, SFstarbucks,SFbaskStadium, SFschools, SFairport, SFtrainStations, SFpet_grooming, SFdog_parks, SFbars_clubs,SFvegan_vegetarian)
SFheatmap

    NEW YORK CITY

In [156]:
NYCheatmap = heat_map("NYCheatmap",new_york_loc, NYC_gj, NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
NYCheatmap

    SEATTLE

In [157]:
STheatmap = heat_map("STheatmap",seattle_loc, ST_gj, STstarbucks,STbaskStadium, STschools, STairport, STtrainStations, STpet_grooming, STdog_parks, STbars_clubs,STvegan_vegetarian)
STheatmap

From what we can see in these heat maps, we can already discard Seattle, as most of the city does not have our requirements.

Although New York is has more heated points, there's also a good sum of places in San Francisco with all our requirements. **Density**

To decide between New York and San Francisco, we can compare the airports and train stations distances.

**NEW YORK VS SAN FRANCISCO**

In [151]:
def travel(map_name, loc, geojson_data, airports_df, train_stations_df):

    ''' Function to get airports and train stations with markers in a city.
    It takes 5 arg:
    - map name
    - location
    - geo json
    - arport df
    - train df.
    '''

    map = folium.Map(location= loc, zoom_start=12)

    airports = gpd.GeoDataFrame(airports_df, geometry=gpd.points_from_xy(airports_df['lon'], airports_df['lat']))
    train_stations = gpd.GeoDataFrame(train_stations_df, geometry=gpd.points_from_xy(train_stations_df['lon'], train_stations_df['lat']))

    for polygon in geojson_data.geometry:

        airports_f = airports[airports.geometry.within(polygon)]

        for _, row in airports_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          icon=folium.Icon(icon='fa-plane', prefix='fa',color='white', icon_color='darkblue'),
                          popup=row['name']).add_to(map) 

    for polygon in geojson_data.geometry:

        train_stations_f = train_stations[train_stations.geometry.within(polygon)]

        for _, row in train_stations_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          tooltip=["station"], 
                          icon=folium.Icon(icon='fa-train', prefix='fa',color='white', icon_color='black'),
                          popup=row['name']).add_to(map) 

    map.save(f"../figures/heatmap{map_name}.html")

    return map

    NEW YORK AIRPORTS AND TRAIN STATIONS

In [158]:
NYtransport_map = travel("NYtransport_map", new_york_loc, NYC_gj, NYCairport, NYCtrainStations)
NYtransport_map

    SAN FRANCISCO AIRPORTS AND TRAIN STATIONS

In [159]:
SFtransport_map = travel("SFtransport_map", san_francisco_loc, SF_gj, SFairport, SFtrainStations)
SFtransport_map

For what we can see in those 2 maps. San Francisco doesn't have a close airport. And the way transport to get there, there are not as many possibilities.

Insted in New York, there are at least 3 closer airports, and a huge multitude of t¡ways to get there, from a lot of places in the city. 

Most of them Manhattan.

This is why, the city I'm going to choose to put my company in New York

## NEW YORK COMPLETE MAP ##

Let's create a complete map with all the requirements to see which neighbourhood is the best to locate the company.

In [167]:
def NY_complete(map_name, loc, geojson_data, starbucks_df,stadium_df, schools_df, airport_df, trainstations_df, pet_grooming_df, dog_parks_df, bars_clubs_df,vegan_vegetarian_df):

    ''' Function to get all the requirements in THE city.
    It takes 12 arg:
    - map name
    - location
    - geo json
    - the 9 df with requirements.
    '''

    map = folium.Map(location= loc, zoom_start=12)

    marker_group_1 = folium.FeatureGroup(name='Starbucks')
    marker_group_2 = folium.FeatureGroup(name='Stadiums')
    marker_group_3 = folium.FeatureGroup(name='Schools')
    marker_group_4 = folium.FeatureGroup(name='airports')
    marker_group_5 = folium.FeatureGroup(name='train stations')
    marker_group_6 = folium.FeatureGroup(name='pet grooming')
    marker_group_7 = folium.FeatureGroup(name='dog parks')
    marker_group_8 = folium.FeatureGroup(name='night clubs')
    marker_group_9 = folium.FeatureGroup(name='vegan restaurants')


    starbucks = gpd.GeoDataFrame(starbucks_df, geometry=gpd.points_from_xy(starbucks_df['lon'], starbucks_df['lat']))
    stadium = gpd.GeoDataFrame(stadium_df, geometry=gpd.points_from_xy(stadium_df['lon'], stadium_df['lat']))
    schools = gpd.GeoDataFrame(schools_df, geometry=gpd.points_from_xy(schools_df['lon'], schools_df['lat']))
    airports = gpd.GeoDataFrame(airport_df, geometry=gpd.points_from_xy(airport_df['lon'], airport_df['lat']))
    train_stations = gpd.GeoDataFrame(trainstations_df, geometry=gpd.points_from_xy(trainstations_df['lon'], trainstations_df['lat']))
    pet_grooming = gpd.GeoDataFrame(pet_grooming_df, geometry=gpd.points_from_xy(pet_grooming_df['lon'], pet_grooming_df['lat']))
    dog_parks = gpd.GeoDataFrame(dog_parks_df, geometry=gpd.points_from_xy(dog_parks_df['lon'], dog_parks_df['lat']))
    bars_clubs = gpd.GeoDataFrame(bars_clubs_df, geometry=gpd.points_from_xy(bars_clubs_df['lon'], bars_clubs_df['lat']))
    vegan_vegetarian = gpd.GeoDataFrame(vegan_vegetarian_df, geometry=gpd.points_from_xy(vegan_vegetarian_df['lon'], vegan_vegetarian_df['lat']))


    for polygon in geojson_data.geometry:

        starbucks_f = starbucks[starbucks.geometry.within(polygon)]

        for _, row in starbucks_f.iterrows():
            folium.Marker([row['lat'], row['lon']],
                            tooltip=["Starbucks"],
                            icon=folium.Icon(icon='fa-coffee', prefix='fa',color='darkgreen', icon_color='white'),
                            popup=row['name']).add_to(marker_group_1)
        
    for polygon in geojson_data.geometry:

        stadium_f = stadium[stadium.geometry.within(polygon)]

        for _, row in stadium_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                            icon=folium.Icon(icon='fa-trophy', prefix='fa',color='darkpurple', icon_color='white'),
                            popup=row['name']).add_to(marker_group_2)
        
    for polygon in geojson_data.geometry:

        schools_f = schools[schools.geometry.within(polygon)]

        for _, row in schools_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                            icon=folium.Icon(icon='fa-graduation-cap', prefix='fa',color='lightblue', icon_color='cadetblue'),
                            popup=row['name']).add_to(marker_group_3)

    for polygon in geojson_data.geometry:

        airports_f = airports[airports.geometry.within(polygon)]

        for _, row in airports_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          icon=folium.Icon(icon='fa-plane', prefix='fa',color='white', icon_color='darkblue'),
                          popup=row['name']).add_to(marker_group_4) 

    for polygon in geojson_data.geometry:

        train_stations_f = train_stations[train_stations.geometry.within(polygon)]

        for _, row in train_stations_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          tooltip=["station"], 
                          icon=folium.Icon(icon='fa-train', prefix='fa',color='white', icon_color='black'),
                          popup=row['name']).add_to(marker_group_5) 
            
    for polygon in geojson_data.geometry:

        pet_grooming_f = pet_grooming[pet_grooming.geometry.within(polygon)]

        for _, row in pet_grooming_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          icon=folium.Icon(icon='fa-paw', prefix='fa',color='white', icon_color='orange'),
                          popup=row['name']).add_to(marker_group_6) 
            
    for polygon in geojson_data.geometry:

        dog_parks_f = dog_parks[dog_parks.geometry.within(polygon)]

        for _, row in dog_parks_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          tooltip=["dog park"], 
                          icon=folium.Icon(icon='fa-thumb-tack', prefix='fa',color='orange', icon_color='white'),
                          popup=row['name']).add_to(marker_group_7) 
            
    for polygon in geojson_data.geometry:

        bars_clubs_f = bars_clubs[bars_clubs.geometry.within(polygon)]

        for _, row in bars_clubs_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          icon=folium.Icon(icon='fa-id-card', prefix='fa',color='black', icon_color='pink'),
                          popup=row['name']).add_to(marker_group_8) 
            
    for polygon in geojson_data.geometry:

        vegan_vegetarian_f = vegan_vegetarian[vegan_vegetarian.geometry.within(polygon)]

        for _, row in vegan_vegetarian_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          tooltip=["vegan restaurant"], 
                          icon=folium.Icon(icon='fa-cutlery', prefix='fa',color='lightgreen', icon_color='white'),
                          popup=row['name']).add_to(marker_group_9) 

    
    marker_group_1.add_to(map)
    marker_group_2.add_to(map)
    marker_group_3.add_to(map)
    marker_group_4.add_to(map)
    marker_group_5.add_to(map)
    marker_group_6.add_to(map)
    marker_group_7.add_to(map)
    marker_group_8.add_to(map)
    marker_group_9.add_to(map)
    
    
    folium.LayerControl(collapsed=False).add_to(map)
    map.save(f"../figures/heatmap{map_name}.html")

    return map

In [168]:
NYselectmap = NY_complete("NYselectmap", new_york_loc, NYC_gj, NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
NYselectmap

For the CEO, looking only at vegan restaurants, and starbucks almost all of Manhattan works. But maybe better Upper West Side, Chelsea adn Union Square.

For clubs it's either Chelsea or the Lower East Side.

For transportation it's Either Chelsea or Midtown.

For Schools it's the Upper West Side and, Werst Village. Chelsea is in between of those 2 neghbourhoods, so not that far.

For Stadiums it's also Chelsea.

For dog parks, the Upper West, Chelsea and Union Square are a good Option.

And for pet grooming it would be Upper West, Upper East and Union Square.


    So as for the data we've got it looks like the best option is Chelsea. Also Chelsea is close to Brooklyn, and there is a Design company there. 

Now that we've got our final decision, let's show in a map how all the requirements would show on the chelsea neighbourhood.

**CHELSEA**

In [178]:
NYC_gj.sample()

,neighborhood,boroughCode,borough,@id,geometry
106,Forest Hills,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.84694 40.73870, -73.84606 40.737..."


In [222]:
def chelsea_select(geojson_data, map_name,starbucks_df, stadium_df, schools_df, airport_df, trainstations_df, pet_grooming_df, dog_parks_df, bars_clubs_df, vegan_vegetarian_df):

    '''Function to get a map with option of selecting markers for Chelsea.
    It takes 11 arg:
    - geo json
    - map name
    - 9 dataframes of requirements.
    '''

    chelsea_neighborhood = geojson_data[geojson_data['neighborhood'] == 'Chelsea']

    map = folium.Map(location=[chelsea_neighborhood.geometry.centroid.y.mean(), chelsea_neighborhood.geometry.centroid.x.mean()], zoom_start=14)

    marker_group_1 = folium.FeatureGroup(name='Starbucks')
    marker_group_2 = folium.FeatureGroup(name='Stadiums')
    marker_group_3 = folium.FeatureGroup(name='Schools')
    marker_group_4 = folium.FeatureGroup(name='airports')
    marker_group_5 = folium.FeatureGroup(name='train stations')
    marker_group_6 = folium.FeatureGroup(name='pet grooming')
    marker_group_7 = folium.FeatureGroup(name='dog parks')
    marker_group_8 = folium.FeatureGroup(name='night clubs')
    marker_group_9 = folium.FeatureGroup(name='vegan restaurants')


    starbucks = gpd.GeoDataFrame(starbucks_df, geometry=gpd.points_from_xy(starbucks_df['lon'], starbucks_df['lat']))
    stadium = gpd.GeoDataFrame(stadium_df, geometry=gpd.points_from_xy(stadium_df['lon'], stadium_df['lat']))
    schools = gpd.GeoDataFrame(schools_df, geometry=gpd.points_from_xy(schools_df['lon'], schools_df['lat']))
    airports = gpd.GeoDataFrame(airport_df, geometry=gpd.points_from_xy(airport_df['lon'], airport_df['lat']))
    train_stations = gpd.GeoDataFrame(trainstations_df, geometry=gpd.points_from_xy(trainstations_df['lon'], trainstations_df['lat']))
    pet_grooming = gpd.GeoDataFrame(pet_grooming_df, geometry=gpd.points_from_xy(pet_grooming_df['lon'], pet_grooming_df['lat']))
    dog_parks = gpd.GeoDataFrame(dog_parks_df, geometry=gpd.points_from_xy(dog_parks_df['lon'], dog_parks_df['lat']))
    bars_clubs = gpd.GeoDataFrame(bars_clubs_df, geometry=gpd.points_from_xy(bars_clubs_df['lon'], bars_clubs_df['lat']))
    vegan_vegetarian = gpd.GeoDataFrame(vegan_vegetarian_df, geometry=gpd.points_from_xy(vegan_vegetarian_df['lon'], vegan_vegetarian_df['lat']))

    starbucks.crs = chelsea_neighborhood.crs
    stadium.crs = chelsea_neighborhood.crs
    schools.crs = chelsea_neighborhood.crs
    airports.crs = chelsea_neighborhood.crs
    train_stations.crs = chelsea_neighborhood.crs
    pet_grooming.crs = chelsea_neighborhood.crs
    dog_parks.crs = chelsea_neighborhood.crs
    bars_clubs.crs = chelsea_neighborhood.crs
    vegan_vegetarian.crs = chelsea_neighborhood.crs


    starbucks_f = starbucks[starbucks.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in starbucks_f.iterrows():
        folium.Marker([row['lat'], row['lon']],
                        tooltip=["Starbucks"],
                        icon=folium.Icon(icon='fa-coffee', prefix='fa',color='darkgreen', icon_color='white'),
                        popup=row['name']).add_to(marker_group_1)
    


    stadium_f = stadium[stadium.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in stadium_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        icon=folium.Icon(icon='fa-trophy', prefix='fa',color='darkpurple', icon_color='white'),
                        popup=row['name']).add_to(marker_group_2)
    


    schools_f = schools[schools.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in schools_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        icon=folium.Icon(icon='fa-graduation-cap', prefix='fa',color='white', icon_color='lightblue'),
                        popup=row['name']).add_to(marker_group_3)



    airports_f = airports[airports.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in airports_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        icon=folium.Icon(icon='fa-plane', prefix='fa',color='white', icon_color='darkblue'),
                        popup=row['name']).add_to(marker_group_4) 



    train_stations_f = train_stations[train_stations.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in train_stations_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        tooltip=["station"], 
                        icon=folium.Icon(icon='fa-train', prefix='fa',color='white', icon_color='black'),
                        popup=row['name']).add_to(marker_group_5) 
        


    pet_grooming_f = pet_grooming[pet_grooming.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in pet_grooming_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        icon=folium.Icon(icon='fa-paw', prefix='fa',color='orange', icon_color='white'),
                        popup=row['name']).add_to(marker_group_6) 
        


    dog_parks_f = dog_parks[dog_parks.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in dog_parks_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        tooltip=["dog park"], 
                        icon=folium.Icon(icon='fa-thumb-tack', prefix='fa',color='white', icon_color='orange'),
                        popup=row['name']).add_to(marker_group_7) 
        


    bars_clubs_f = bars_clubs[bars_clubs.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in bars_clubs_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        icon=folium.Icon(icon='fa-id-card', prefix='fa',color='black', icon_color='pink'),
                        popup=row['name']).add_to(marker_group_8) 
        


    vegan_vegetarian_f = vegan_vegetarian[vegan_vegetarian.geometry.within(chelsea_neighborhood.unary_union)]

    for _, row in vegan_vegetarian_f.iterrows():
        folium.Marker([row['lat'], row['lon']], 
                        tooltip=["vegan restaurant"], 
                        icon=folium.Icon(icon='fa-cutlery', prefix='fa',color='lightgreen', icon_color='white'),
                        popup=row['name']).add_to(marker_group_9) 
        
    icon = folium.Icon(
                color = "red",
                icon_color = "white",
                icon = "fa-location-arrow",
                prefix = "fa"
            )
    new_marker = folium.Marker(location = [40.74739,-73.99325], icon=icon, z_index_offset=1000)
    new_marker.add_to(map)

    
    marker_group_1.add_to(map)
    marker_group_2.add_to(map)
    marker_group_3.add_to(map)
    marker_group_4.add_to(map)
    marker_group_5.add_to(map)
    marker_group_6.add_to(map)
    marker_group_7.add_to(map)
    marker_group_8.add_to(map)
    marker_group_9.add_to(map)
    
    folium.Circle(location=[40.74739,-73.99325], fill_color='purple', radius=750, weight=2, color='purple', popup='Desired Area').add_to(map)
    folium.LayerControl(collapsed=False).add_to(map)
    map.save(f"../figures/heatmap{map_name}.html")

    return map


In [204]:
chelsea_map =  chelsea_select(NYC_gj,"Chelsea_map",NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
chelsea_map

/var/folders/ns/rjvmq7zn3cggtjmkkj8bhfsc0000gn/T/ipykernel_48321/2194234581.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map = folium.Map(location=[chelsea_neighborhood.geometry.centroid.y.mean(), chelsea_neighborhood.geometry.centroid.x.mean()], zoom_start=14)


The 28th with 7th Ave looks like a great location. Its 2 streets for coffe, 3 for train, 3 for vegan restaurants and a lot more that doesn't appear on the map. It's next to Madison Square Park, Herald, Square. There are a lot of Metro Stations, restaurants, rooftops, and a hole lot more starbucks. It's one of the bests locations in the city for sure. 

Now that we've chosen a possible location let's do a more thorough search on the area.

emplenar mes els restaurants, els starbucks i els clubs

In [205]:
CSstarbucks = mongodb_to_df("Chelsea_starbucks")
CSbaskStadium = mongodb_to_df("Chelsea_stadiums")
CSschools = mongodb_to_df("Chelsea_school")
CSschools = mongodb_to_df("Chelsea_airport")
CStrainStations = mongodb_to_df("Chelsea_train_stations")
CSCSdog_parks = mongodb_to_df("Chelsea_dog_park")
CSbars_clubs = mongodb_to_df("Chelsea_bars_clubs")
CSvegan_vegetarian = mongodb_to_df("Chelsea_vegan_vegetarian")

In [223]:
detail_chelsea_map =  chelsea_select(NYC_gj,"Detail_Chelsea_map",CSstarbucks,CSbaskStadium, CSschools, NYCairport, NYCtrainStations, NYCpet_grooming, CSCSdog_parks, CSbars_clubs,CSvegan_vegetarian)
detail_chelsea_map

/var/folders/ns/rjvmq7zn3cggtjmkkj8bhfsc0000gn/T/ipykernel_48321/3503930161.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map = folium.Map(location=[chelsea_neighborhood.geometry.centroid.y.mean(), chelsea_neighborhood.geometry.centroid.x.mean()], zoom_start=14)


Doing a more thorough investigation, we can see that there's a bunch of schools right next to the chosen location, and dog parks. Also Starbucks only a minute away. More variety of vegan restaurants, and a hole lot more of night clubs. That on top of the already found stadiums, train stations and ways to get to the airport. All in all, I think we've got our perfect location. 

All that within the borders of the Chelsea Neighborhood geo json.